In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
from ScrimLog import *

In [2]:
scrim = ScrimLog('20210326_02_TAL_Volskaya Industries.csv').get_df_FinalStat()
display(scrim)

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

MatchId      Map                  Section  TF_order
20210326_02  Volskaya Industries  0        1.0        -0.002665
                                           2.0        -0.002665
                                           3.0        -0.002665
                                           4.0        -0.002665
                                  1        1.0        -0.002665
                                           2.0        -0.002665
                                           3.0        -0.002665
                                           4.0        -0.002665
                                  2        1.0        -0.002665
                                           2.0        -0.002665
                                  3        1.0        -0.002665
                                           2.0        -0.002665
                                           3.0        -0.002665
                                           4.0        -0.002665
Name: DominanceIndex, dtype: float64

HeroDamageDealt  \
MatchId     Map                 Section Point Hero     Player    RoundName Team Timestamp TF_order                    
20210326_02 Volskaya Industries 0       0.0   Zarya    BiaNcA    Defense   NYE  0.00      1.0                  0.00   
                                                                                          2.0                  0.00   
                                                                                          3.0                  0.00   
                                                                                          4.0                  0.00   
                                              Brigitte Friday    Defense   NYE  0.00      1.0                  0.00   
...                                                                                                             ...   
                                3       0.0   Winston  Kellan    Defense   TAL  862.28    4.0               6683.47   
                                              Ana      vigilante Defense   TAL  862.28    1.0               3105.92   
                                                                                          2.0               3105.92   
                                                                                          3.0               3105.92   
                                                                                          4.0               3105.92   

                                                                                                    BarrierDamageDealt  \
MatchId     Map                 Section Point Hero     Player    RoundName Team Timestamp TF_order                       
20210326_02 Volskaya Industries 0       0.0   Zarya    BiaNcA    Defense   NYE  0.00      1.0                     0.00   
                                                                                          2.0                     0.00   
                                                                                          3.0                     0.00   
                                                                                          4.0                     0.00   
                                              Brigitte Friday    Defense   NYE  0.00      1.0                     0.00   
...                                                                                                                ...   
                                3       0.0   Winston  Kellan    Defense   TAL  862.28    4.0                  1285.13   
                                              Ana      vigilante Defense   TAL  862.28    1.0                   767.88   
                                                                                          2.0                   767.88   
                                                                                          3.0                   767.88   
                                                                                          4.0                   767.88   

                                                                                                    DamageBlocked  \
MatchId     Map                 Section Point Hero     Player    RoundName Team Timestamp TF_order                  
20210326_02 Volskaya Industries 0       0.0   Zarya    BiaNcA    Defense   NYE  0.00      1.0                 0.0   
                                                                                          2.0                 0.0   
                                                                                          3.0                 0.0   
                                                                                          4.0                 0.0   
                                              Brigitte Friday    Defense   NYE  0.00      1.0                 0.0   
...                                                                                                           ...   
                             

In [3]:
def update_FinalStat(save_dir='G:/공유 드라이브/NYXL Scrim Log/FinalStat/'):
    filepath = 'G:/공유 드라이브/NYXL Scrim Log/Csv/'
    filelist = os.listdir(filepath)
    csv_filelist = [x for x in filelist if x.endswith('.csv')]
    updated_csv = 'FilesUpdated.txt'

    f = open(filepath + updated_csv, 'r+')
    lines = f.readlines()
    updated_filelist = []

    for line in lines:
        updated_filelist.append(line.replace('\n', ''))

    csv_filelist_to_export = list(set(csv_filelist) - set(updated_filelist))

    # export to csv in FinalStat folder
    for filename in csv_filelist_to_export:
        ScrimLog(filename).export_to_csv()

        f.write(filename+'\n')

    f.close()

update_FinalStat()

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

MatchId      Map               Section  TF_order
20210326_01  Temple of Anubis  0        1.0         0.006032
                                        2.0         0.006032
                                        3.0         0.006032
                                        4.0         0.006032
                               1        1.0         0.006032
                                        2.0         0.006032
                                        3.0         0.006032
                                        4.0         0.006032
                                        5.0         0.006032
                                        6.0         0.006032
                                        7.0         0.006032
Name: DominanceIndex, dtype: float64

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

MatchId      Map          Section  TF_order
20210325_02  Eichenwalde  0        1.0        -0.008968
                                   2.0        -0.008968
                                   3.0        -0.008968
                                   4.0        -0.008968
                                   5.0        -0.008968
                                   6.0        -0.008968
                                   7.0        -0.008968
                          1        1.0        -0.008968
                                   2.0        -0.008968
                                   3.0        -0.008968
                                   4.0        -0.008968
                                   5.0        -0.008968
                                   6.0        -0.008968
                                   7.0        -0.008968
                                   8.0        -0.008968
                                   9.0        -0.008968
Name: DominanceIndex, dtype: float64

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

...

MatchId      Map             Section  TF_order
20210326_01  Blizzard World  0        1.0         0.001345
                                      2.0         0.001345
                                      3.0         0.001345
                                      4.0         0.001345
                                      5.0         0.001345
                                      6.0         0.001345
                                      7.0         0.001345
                                      8.0         0.001345
                                      9.0         0.001345
                             1        1.0         0.001345
                                      2.0         0.001345
                                      3.0         0.001345
                                      4.0         0.001345
                                      5.0         0.001345
                                      6.0         0.001345
                                      7.0         0.001345
         

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

MatchId      Map         Section  TF_order
20210325_01  King's Row  0        1.0        -0.014994
                                  2.0        -0.014994
                                  3.0        -0.014994
                                  4.0        -0.014994
                                  5.0        -0.014994
                                  6.0        -0.014994
                         1        1.0        -0.014994
                                  2.0        -0.014994
                                  3.0        -0.014994
                                  4.0        -0.014994
                                  5.0        -0.014994
                                  6.0        -0.014994
                                  7.0        -0.014994
Name: DominanceIndex, dtype: float64

d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stat_level_col['new_col'] = stat_level_col['HeroDamageDealt'] + stat_level_col['BarrierDamageDealt']
C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\pandas\core\frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
d:\owl-data\Project\project 5_scrim stat log\TraditionalStat.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

MatchId      Map     Section  TF_order
20210326_01  Dorado  0        1.0        -0.014168
                              2.0        -0.014168
                              3.0        -0.014168
                              4.0        -0.014168
                              5.0        -0.014168
                              6.0        -0.014168
                              7.0        -0.014168
                              8.0        -0.014168
                              9.0        -0.014168
                              10.0       -0.014168
                     1        1.0        -0.014168
                              2.0        -0.014168
                              3.0        -0.014168
                              4.0        -0.014168
                              5.0        -0.014168
                              6.0        -0.014168
                              7.0        -0.014168
                              8.0        -0.014168
                              9.0        -0

In [ ]:
# df_peth_FB = PETH(df_init=df_FinalStat, event_name='FinalBlows/s', period=5)
# df_peth_FB = df_peth_FB.get_PETH(of='Tracer').groupby(by=['MatchId', 'Map', 'Section', 'num_event', 'Timestamp', 'Team', 'Player', 'Hero']).sum()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='Cooldown1%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='Cooldown2%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='CooldownSecondaryFire%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='CooldownCrouching%', data=df_peth_FB, hue='Hero')
# plt.show()

In [ ]:
'''
PETH 개념을 거꾸로 가져가서 몇 가지 event가 동시에 or interval 이내에 일어났을 때를 기준으로 (t=0) 'deaths/s', 'final blows/s' 등의 event 확률 histogram 보여주는 기능 추가
'''